In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

<h2> Učitavanje skupa podataka i generalni uvid u podatke </h2>

In [2]:
# učitavanje podataka
mydateparser = lambda c: pd.to_datetime(c, format='%d.%m.%Y', errors='coerce')
df = pd.read_csv('training_dataset_enc.csv', parse_dates=['DATUM_IZVJESTAVANJA', 'PLANIRANI_DATUM_ZATVARANJA', 'DATUM_OTVARANJA', 'DATUM_ZATVARANJA'], date_parser=mydateparser)

In [3]:
# brisanje nepotrebnih stupaca 
del df['Unnamed: 0']; del df['Unnamed: 0.1']; 

In [10]:
df.shape

(5193124, 17)

In [4]:
df.head()

,DATUM_IZVJESTAVANJA,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,DATUM_ZATVARANJA,UGOVORENI_IZNOS,STANJE_NA_KRAJU_PRETH_KVARTALA,STANJE_NA_KRAJU_KVARTALA,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,2013-12-31,412140,7146814,2012-08-27,2018-04-30,NaT,629155.97,639373.81,641562.01,2,1420,TM0109,L,1.50,A,22,N
1,2013-12-31,772139,5521579,2010-06-11,2017-06-30,NaT,46000.00,28481.94,26887.19,1,1410,FL0801,A,9.70,B,38,N
2,2013-12-31,481985,7443823,2013-01-23,2018-02-28,NaT,44679.50,44908.40,45062.09,2,1550,FD0100,L,3.10,A,37,N
3,2013-12-31,139014,6511639,2011-09-16,2016-09-30,NaT,29959.15,19691.64,18285.89,2,1410,FL0801,A,7.95,B,49,N
4,2013-12-31,952566,6143123,2011-05-23,2014-05-31,NaT,100000.00,24797.93,15685.33,1,1410,FL0801,A,9.70,B,35,N


In [5]:
df.describe()

,KLIJENT_ID,OZNAKA_PARTIJE,UGOVORENI_IZNOS,STANJE_NA_KRAJU_PRETH_KVARTALA,STANJE_NA_KRAJU_KVARTALA,VALUTA,VRSTA_KLIJENTA,VISINA_KAMATE,STAROST
count,5.193124e+06,5.193124e+06,5.193124e+06,4.772056e+06,5.193124e+06,5.193124e+06,5.193124e+06,5.141422e+06,5.193124e+06
mean,7.207398e+05,7.152089e+06,2.554007e+05,1.309054e+05,1.181028e+05,1.682643e+00,1.426380e+03,5.562767e+00,4.141305e+01
std,1.160615e+07,8.765659e+06,3.784332e+06,1.950900e+06,1.829336e+06,8.221024e-01,6.029030e+01,3.760118e+00,2.045255e+02
min,1.002000e+03,1.630070e+05,0.000000e+00,-5.981000e+01,-5.981000e+01,1.000000e+00,1.110000e+03,-4.000000e-03,-7.994000e+03
25%,3.708580e+05,4.275223e+06,2.956673e+04,6.897520e+03,4.032975e+02,1.000000e+00,1.410000e+03,2.350000e+00,3.500000e+01
50%,6.930290e+05,6.603912e+06,6.422751e+04,3.339354e+04,2.673879e+04,2.000000e+00,1.410000e+03,6.480000e+00,4.600000e+01
75%,1.033071e+06,9.142953e+06,1.150000e+05,8.280279e+04,7.627030e+04,2.000000e+00,1.410000e+03,8.740000e+00,5.700000e+01
max,9.991411e+09,1.136257e+08,6.055048e+08,6.105662e+08,6.105662e+08,5.000000e+00,1.610000e+03,8.080000e+01,9.080000e+02


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5193124 entries, 0 to 5193123
Data columns (total 17 columns):
DATUM_IZVJESTAVANJA               datetime64[ns]
KLIJENT_ID                        int64
OZNAKA_PARTIJE                    int64
DATUM_OTVARANJA                   datetime64[ns]
PLANIRANI_DATUM_ZATVARANJA        datetime64[ns]
DATUM_ZATVARANJA                  datetime64[ns]
UGOVORENI_IZNOS                   float64
STANJE_NA_KRAJU_PRETH_KVARTALA    float64
STANJE_NA_KRAJU_KVARTALA          float64
VALUTA                            int64
VRSTA_KLIJENTA                    int64
PROIZVOD                          object
VRSTA_PROIZVODA                   object
VISINA_KAMATE                     float64
TIP_KAMATE                        object
STAROST                           int64
PRIJEVREMENI_RASKID               object
dtypes: datetime64[ns](4), float64(4), int64(5), object(4)
memory usage: 673.5+ MB


Ciljna značajka je varijabla 'PRIJEVREMENI_RASKID'. 

Primjećujemo da u stupcu 'STAROST' postoje ekstremne vrijednosti kao što su negativne vrijednosti i vrijednosti veće od 900. Rečeno nam je da to znači da podatak nije dostupan, tj. vrlo vjerojatno se radi o pravnoj osobi pa ćemo postaviti te vrijednosti na -1.

In [ ]:
df['STAROST'].values[df['STAROST'].values >= 900] = -1
df['STAROST'].values[df['STAROST'].values < 0] = -1

In [24]:
# istražujemo kategoričke varijable 
columns = ['VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA','TIP_KAMATE', 'PRIJEVREMENI_RASKID'];
for column_name in columns:
    print(column_name,': ', df[column_name].unique())

VALUTA :  [2 1 3 4 5]
VRSTA_KLIJENTA :  [1420 1410 1550 1610 1430 1120 1110 1320 1210 1510 1310 1330 1350]
PROIZVOD :  ['TM0109' 'FL0801' 'FD0100' 'FL1301' 'FL1100' 'FL1200' 'FL1201' 'FL0802'
 'FL1202' 'FL1306' 'FL1309' 'FL0900' 'FL0803' 'FL1400' 'FL1600' 'FL0804'
 'FL1500' 'FL1310' 'FD0200' 'TM0104' 'FS0200' 'FL1304' 'FI0500' 'FP0100'
 'FL0201' 'FL0202' 'FL0901' 'FL0600' 'TM0102' 'TM0500' 'FE0100' 'FE0200']
VRSTA_PROIZVODA :  ['L' 'A']
TIP_KAMATE :  ['A' 'B' 'C' 'D']
PRIJEVREMENI_RASKID :  ['N' 'Y']


<h3> (Ne)Balansiranost skupa podataka </h3>

In [18]:
prekidi = df.loc[df['PRIJEVREMENI_RASKID']=='Y']
prekidi_group = prekidi.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])
len1 = len(prekidi_group);
print('Broj prijevremenih raskida: ', len1)

sve_grupe = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])
len2 = len(sve_grupe);
print('Ukupan broj ugovora: ', len2)
print('Postotak prijevremenih raskida u datasetu: ', len1/len2 )

Broj prijevremenih raskida:  160155
Ukupan broj ugovora:  950239
Postotak prijevremenih raskida u datasetu:  0.16854180895543122


Kako je postotak prijevremenih raskida u skupu podataka otprilike 17%, zaključujemo da je dataset nebalansiran pa moramo to uzeti u obzir pri odabiru ML modela.

<h3>Missing values </h3>

In [12]:
df.isnull().sum()

DATUM_IZVJESTAVANJA                     0
KLIJENT_ID                              0
OZNAKA_PARTIJE                          0
DATUM_OTVARANJA                         0
PLANIRANI_DATUM_ZATVARANJA         421129
DATUM_ZATVARANJA                  3978638
UGOVORENI_IZNOS                         0
STANJE_NA_KRAJU_PRETH_KVARTALA     421068
STANJE_NA_KRAJU_KVARTALA                0
VALUTA                                  0
VRSTA_KLIJENTA                          0
PROIZVOD                                0
VRSTA_PROIZVODA                         0
VISINA_KAMATE                       51702
TIP_KAMATE                              0
STAROST                                 0
PRIJEVREMENI_RASKID                     0
dtype: int64

In [14]:
df.isnull().sum() ['DATUM_ZATVARANJA'] / df.shape[0]

0.7661357595158521

Budući da je otprilike 77% podataka u stupcu 'DATUM_ZATVARANJA' nepoznato, obrisat ćemo taj stupac.

In [15]:
del df['DATUM_ZATVARANJA']

In [33]:
tip_kamate_kredita = df.groupby(['TIP_KAMATE', 'VRSTA_PROIZVODA', 'KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA' ])['VISINA_KAMATE']

In [34]:
tip_kamate_kredita.mean()

TIP_KAMATE  VRSTA_PROIZVODA  KLIJENT_ID  OZNAKA_PARTIJE  DATUM_OTVARANJA
A           A                1013        11378209        2016-06-07         1.900000
                                         12480085        2017-03-07         2.100000
                             1061        349107          2005-02-28         5.000000
                             1063        9738470         2014-04-24         4.236842
                             1100        168604          2007-10-29         4.000000
                                         7058587         2012-06-28         4.000000
                                         11274030        2015-10-05              NaN
                             1103        1213645         2006-08-31         9.250000
                             1115        10342567        2015-05-27         8.950000
                             1116        7534409         2013-03-11         8.450000
                             1123        1309841         2003-12-03         3

In [32]:
tip_kamate_kredita.min()

TIP_KAMATE  KLIJENT_ID  OZNAKA_PARTIJE  DATUM_OTVARANJA
A           1002        6403355         2011-07-07         2.10
                        6426427         2011-08-04         2.55
                        6501523         2011-09-19         2.35
                        6548855         2011-10-20         1.60
                        6592319         2011-11-18         1.60
                        6643516         2011-12-28         3.00
                        6726203         2012-02-07         3.50
                        7146985         2012-09-04         1.70
                        7220379         2012-10-01         1.30
                        7288118         2012-11-09         1.00
                        9956379         2014-08-20         1.00
                        12254464        2016-10-28         1.00
            1013        5903781         2011-02-15          NaN
                        5916726         2011-02-21          NaN
                        5916741         2011-02-